In [1]:
import os

In [2]:
%pwd

'c:\\Users\\jatin\\Desktop\\internship_project\\Alzimer_disease_detection\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\jatin\\Desktop\\internship_project\\Alzimer_disease_detection'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [6]:
from Alzimer_disease_classifier.constants import *
from Alzimer_disease_classifier.utils.common import read_yaml, create_directories
from Alzimer_disease_classifier.entity.config_entity import PrepareBaseModelConfig
import tensorflow as tf



In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifact_root])

    def get_training_config(self)->TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = Path(self.config.data_ingestion.root_dir)
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [8]:
from zipfile import ZipFile
import time

In [9]:
import os 
import urllib.request as request
import tensorflow as tf
import time

In [10]:
class Training:
   def __init__(self, config: TrainingConfig):
        self.config = config

   def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

   def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation='nearest',
            
        )

        validation_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
           **datagenerator_kwargs
        )

        self.valid_generator = validation_datagenerator.flow_from_directory(
           directory = self.config.training_data,
           subset = "validation",
           shuffle = False,
           **dataflow_kwargs
        )
        if self.config.params_is_augmentation:
           train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
              rotation_range = 40,
              horizontal_flip= True,
              width_shift_range = 0.2,
              height_shift_range = 0.2,
              shear_range = 0.2,
              zoom_range = 0.2,
              **datagenerator_kwargs
           )
        else:
           train_datagenerator = validation_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
           directory = self.config.training_data,
           subset = "training",
           shuffle = True,
           **dataflow_kwargs
        )

   @staticmethod
   def save_model(path:Path, model: tf.keras.Model):
       model.save(path)

   def train(self):
      self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
      self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

      self.model.fit(
         self.train_generator,
         epochs = self.config.params_epochs,
         steps_per_epoch = self.steps_per_epoch,
         validation_steps = self.validation_steps,
         validation_data = self.valid_generator,

      )
      self.save_model(
         path = self.config.trained_model_path,
         model = self.model
      )

In [11]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()

except Exception as e:
    raise e

Found 43 images belonging to 2 classes.
Found 176 images belonging to 2 classes.
4/4 [==============================] - 29s 7s/step - loss: 15.2353 - accuracy: 0.5221 - val_loss: 29.2191 - val_accuracy: 0.5500


c:\Users\jatin\Desktop\internship_project\Alzimer_disease_detection\alz_venv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [52]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
import logging
from tensorflow.keras.models import load_model

model_path = os.path.join("artifacts", "training", "model.keras")
model = load_model(model_path)
  # Assuming the model is already trained and available
file_name = r'C:\Users\jatin\Desktop\internship_project\Alzimer_disease_detection\artifacts\data_ingestion\mild\mild_35.jpg'
test_image = load_img(file_name, target_size=(224, 224))
test_image = img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
test_image = test_image / 255.0  # Normalize if required
logging.debug("Image loaded and preprocessed successfully.")

# Make prediction
prediction_prob = model.predict(test_image)
result = np.argmax(prediction_prob, axis=1)
logging.info(f"Raw prediction output: {prediction_prob}")
logging.info(f"Argmax result: {result}")

# Decode prediction
if result[0] == 1:
    print("no Alzheimer's Disease")
else:
    print("Alzheimer's Disease")

1/1 [==============================] - 0s 281ms/step
Alzheimer's Disease


In [36]:
def verify_model_trained(model_path, test_image):
    model = tf.keras.models.load_model(model_path)
    
    # Check 1: Basic prediction
    pred = model.predict(test_image)
    print(f"Prediction output: {pred}")
    
    # Check 2: Weight means
    first_conv_weights = model.layers[1].get_weights()[0]
    print(f"First conv layer weights mean: {np.mean(first_conv_weights):.4f}")
    
    # Check 3: Output confidence
    if pred.shape[1] > 1:  # Multiclass
        confidence = tf.reduce_max(tf.nn.softmax(pred))
    else:  # Binary
        confidence = tf.sigmoid(pred)[0]
    print(f"Prediction confidence: {confidence:.2%}")

# Usage
test_sample = tf.expand_dims(test_image[0], 0)
verify_model_trained(model_path, test_sample)

1/1 [==============================] - 0s 289ms/step
Prediction output: [[1.00000000e+00 1.06665896e-19]]
First conv layer weights mean: -0.0024
Prediction confidence: 73.11%


In [43]:
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     